In [1]:
import json
import numpy as np
import pandas as pd
import csv
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.base import clone
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import auc, roc_curve, roc_auc_score
from keras.models import model_from_json

sns.set_style("darkgrid")

### We can now use the model to predict comps not played as much with different carries
We focus on the mock data of knights that we created and recall that Kayle was the main non-knight carry but what about other possible carries like Lucian, Akshan, Karma or Teemo? All of these carry-worthy heroes do not show up (with Garen and Galio) too much in the data (typically single digits)

In [2]:
mock_data = pd.read_pickle("mock_data_encoded.pkl")

In [3]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [4]:
mock_data_input = mock_data.iloc[:,:-4]

In [5]:
list(mock_data_input.columns)

['Set5_Assassin',
 'Set5_Brawler',
 'Set5_Cavalier',
 'Set5_Forgotten',
 'Set5_Ironclad',
 'Set5_Legionnaire',
 'Set5_Nightbringer',
 'Set5_Redeemed',
 'Set5_Sentinel',
 'Set5_Skirmisher',
 'Set5_Dawnbringer',
 'Set5_Invoker',
 'Set5_Abomination',
 'Set5_Knight',
 'Set5_Hellion',
 'Set5_Ranger',
 'Set5_Spellweaver',
 'Set5_Cannoneer',
 'Set5_Revenant',
 'Set5_Draconic',
 'Set5_Mystic',
 'Set5_Renewer',
 'Number of Chromatic',
 'Number of Gold',
 'Number of Silver',
 'Number of Bronze',
 'Synergy Contest_1',
 'Synergy Contest_2',
 'Synergy Contest_3',
 'Vertical Synergies_1',
 'Vertical Synergies_2',
 'Primary_Carry_Tier',
 'Primary_Carry_1',
 'Primary_Carry_2',
 'Primary_Carry_3',
 'Primary_Carry_4',
 'Primary_Carry_5',
 'Primary_Carry_6',
 'Primary_Carry_7',
 'Primary_Carry_8',
 'Primary_Carry_9',
 'Primary_Carry_10',
 'Primary_Carry_11',
 'Primary_Carry_12',
 'Primary_Carry_13',
 'Primary_Carry_14',
 'Primary_Carry_15',
 'Primary_Carry_16',
 'Primary_Carry_17',
 'Primary_Carry_18',
 

In [6]:
mock_data_prob = model.predict(np.asarray(mock_data_input[list(mock_data_input.columns)].values).astype('float32'))
mock_top_4_prob = mock_data_prob[:,1]
mock_pred = [np.argmax(x) for x in mock_data_prob]

In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [8]:
mock_data_unencoded = pd.read_pickle("mock_data.pkl")
info = mock_data_unencoded.iloc[:,-12:-6]
info["Top 4 Prediction"] = mock_pred
info["Top 4 Prob"] = mock_top_4_prob

In [9]:
info[info["Top 4 Prediction"] == 1].sort_values(by = ["Top 4 Prob"], ascending=False)

,Primary_Carry_Tier,Primary_Carry,Secondary_Carry_Tier,Secondary_Carry,Third_Carry_Tier,Third_Carry,Top 4 Prediction,Top 4 Prob
330,2,TFT5_Galio,2,TFT5_Garen,2,TFT5_Heimerdinger,1,0.849770
138,2,TFT5_Galio,2,TFT5_Garen,2,TFT5_Akshan,1,0.847532
90,2,TFT5_Galio,2,TFT5_Garen,2,TFT5_Akshan,1,0.798532
140,2,TFT5_Garen,2,TFT5_Galio,2,TFT5_Akshan,1,0.795080
282,2,TFT5_Galio,2,TFT5_Garen,2,TFT5_Draven,1,0.787975
92,2,TFT5_Garen,2,TFT5_Galio,2,TFT5_Akshan,1,0.775484
139,2,TFT5_Galio,2,TFT5_Akshan,2,TFT5_Garen,1,0.769930
114,1,TFT5_Galio,2,TFT5_Garen,2,TFT5_Akshan,1,0.747274
332,2,TFT5_Garen,2,TFT5_Galio,2,TFT5_Heimerdinger,1,0.744235
382,2,TFT5_Karma,2,TFT5_Galio,2,TFT5_Garen,1,0.737936
